# Homework 5 - Visit the Wikipedia hyperlinks graph!

In this assignment we perform an analysis of the Wikipedia Hyperlink graph. In particular, given extra information about the
categories to which an article belongs to, we are curious to rank the articles according to some criteria.

### [RQ1] Build the graph G = (V, E), where V is the set of articles and E the hyperlinks among them, and provide its basic information:
- If it is direct or not
- The number of nodes
- The number of edges
- The average node degree. Is the graph dense?

In [1]:
import pandas as pd
import json
from tqdm import tqdm
from collections import defaultdict
from collections import deque
import networkx as nx
import math
import statistics
import heapq 
import json

Observing the file **wiki-topcats-reduced.txt**, we can notice the existence of a couple of nodes (107 e 104) for which it exists an edge from 107 to 104 and from 104 to 107. This make us realize that the graph is direct, otherwise, it would be useless the existence of one of the two edges. We thought about this proof: let's suppose that the graph is not direct. Then it can not exists a couple of nodes $a$ and $b$ such that exists an edge $a \rightarrow b$ and an edge $b \rightarrow a$. If exists a couple of nodes with this property than the graph is direct. To prove this we opened the file **reduced** and save it in a dataframe.

In [2]:
reduced = pd.read_csv('wiki-topcats-reduced.txt', sep = '\t', names = ['source', 'destination'])

In [3]:
reduced.head()

,source,destination
0,52,401135
1,52,1069112
2,52,1163551
3,62,12162
4,62,167659


In [4]:
len(reduced)

2645247

Now we for each couple $(a,b)$ we add an edge $b\rightarrow a$. In this way for each couple $(a,b)$ exists an edge $b\rightarrow a$ and $a\rightarrow b$. So if the are duplicates, then already exist a couple of nodes with this property, so the graph will be direct.
To do this we invert the two columns, concatenating the two dataframe, and then we remove the duplicates.

In [5]:
reduced_inv=reduced.reindex(columns=['destination','source'])
reduced_inv.columns=['source','destination']

In [6]:
print('Number of edges of the graph with possible duplicates: ',len(pd.concat([reduced,reduced_inv])))
print('Number of edges of the graph without duplicates: ',len(pd.concat([reduced,reduced_inv]).drop_duplicates()))

Number of edges of the graph with possible duplicates:  5290494
Number of edges of the graph without duplicates:  4348125


We can see that the number of edges decrease. So the graph is direct.

Now we create our graph, using a dictonary, in which the keys are the nodes and the values the list of nodes to which the node is linked by the edge. We also build the "inverse_graph", that we will need later. In the inverse graph the keys are the list of nodes and the values are the nodes from which the edge starts.  

In [7]:
direct_graph=defaultdict(list)
inverse_graph=defaultdict(list)
with open('wiki-topcats-reduced.txt') as f: 
    for line in tqdm(f):
        l=list(map(int,line.split()))
        direct_graph[l[0]].append(l[1])
        inverse_graph[l[1]].append(l[0])
#complete the graph
for key in inverse_graph.keys():
    if key not in direct_graph:
        direct_graph[key]=[]

2645247it [00:11, 236031.94it/s]


To compute the total number number of nodes, we make the union between the  set of the keys of the direct_graph and the set of the keys of the inverse_graph.
The number of edge is the sum of the lengths of the lists in values of the direct graph. 

In [8]:
V=len(direct_graph.keys())
E=sum(map(len,direct_graph.values()))
D=E/(V*(V-1))
print('Number of nodes: ',V)
print('Number of edges: ',E)
print('Average node degree (IN): ',sum(map(len,inverse_graph.values()))/V)
print('Average node degree (OUT): ',E/V)
print('Density ratio: ', D)

Number of nodes:  461193
Number of edges:  2645247
Average node degree (IN):  5.735661642739591
Average node degree (OUT):  5.735661642739591
Density ratio:  1.2436602635647606e-05


From the in-degree and the density ratio we computed, we can say that the graph is not dense! Now we create another dictionary that has for keys the categories and values the list of articles that are in the category. After we will remove the categories that have less than 3500 argument.

In [9]:
categories={}
with open('wiki-topcats-categories.txt') as f:
    for line in f:
        l=line.split()
        categories[l[0][9:-1]]=list(map(int,l[1::]))

In [10]:
graph=categories.copy()
for key in categories:
    if len(categories[key])<=3500:
        del(graph[key]) 

Now we remove the useless node, that are the nodes without outcoming edges and incoming edges

In [11]:
clean_cat=graph.copy()
vertex=set(direct_graph.keys()).union(set(inverse_graph.keys()))
for key in graph:
    clean_cat[key]=list(set(graph[key]).intersection(vertex))

Now we add to the graph, the nodes that don't have outgoing edges.

### [RQ2] Given a category  $C_0 = \{article_1, article_2, \dots \}$ as input we want to rank all of the nodes in V according to the following criteria: 
Obtain a block-ranking, where the blocks are represented by the categories. In particular, we want:

$$block_{RANKING}=\left[\array{C_0,\\C_1,\\ \cdots \\C_C }\right]$$ 

Each category $C_i$ corresponds to a list of nodes.

In the following chuncks of code, we implement some tools that we need for the analysis. The first tool is the BFS (Breadth-first-search), that we need for explore all the graph. Given a graph and a starting node (root) it returns the search_tree that is a tree that for each node has for sons, the closer nodes. We compute the distances between a node and the root during the bfs

In [12]:
def bfs(graph, root):
    seen, queue = set([root]), deque([root])
    distances=defaultdict(lambda: math.inf)
    distances[root] = 0
    while queue:
        vertex = queue.popleft()
        d = distances[vertex]
        for node in graph[vertex]:
            if node not in seen:
                seen.add(node)
                queue.append(node)
                distances[node] = d+1
    return(distances)

With the function *merge_distances* we can store all the shortest path from each node of the category $C_0$ to each nodes of each category $C_i$. At the end, we will get a dictionary where the keys are the nodes of the graph and the values are the lists of distances between the node and the nodes of $C_0$ 

In [13]:
def merge_dist(d_min,d):
    for key in d_min:
        d_min[key].append(d[key])
    return d_min

Since we want to rank the categories using the following criteria: 

$$distance(C_0,C_i)=median(ShortestPath(C_0,C_i))$$

we compute the median on the lists of lenghts of all shortest path from $C_0$ to $C_i \quad \forall i\in\{1,\cdots,C\}$  

In [14]:
def compute_median(l):
    return statistics.median(l)

In [15]:
C0=clean_cat['Year_of_birth_unknown']
min_dist={i:[] for i in direct_graph.keys()}
for root in tqdm(C0):
    dist=bfs(direct_graph, root)
    min_dist=merge_dist(min_dist, dist)

100%|██████████| 2536/2536 [1:03:45<00:00,  1.61it/s]


Once calculated all length of all shortest path, we store it in a dictionary, build in this way:

$$\textit{short_path}=\{C_i: [\textit{distances between each nodes of $C_0$ and $C_i$}]\}\quad \forall i \in \{0,\cdots,C\}$$

In [16]:
short_path=defaultdict(list)
for cat in tqdm(clean_cat):
    nodes=clean_cat[cat]
    for key in nodes:
        short_path[cat]+=min_dist[key] 

100%|██████████| 35/35 [12:22<00:00, 18.40s/it]


Now we rank the categories using the value of the median. We store different tuples, made by:

$$(\textit{median}_i,C_i)$$

in a min_heap. This because the min_heap has the minimum value of the root, so it's easy to sort this list of tuples, using the heapsort algorythm. We observe that there are a lot of nodes with distance equal to $\infty$, this probabily because the graph is direct, so there are different nodes that have only outgoing edges, and also the graph is not dense.

In [23]:
ranking=[]
for cat in tqdm(short_path):
    heapq.heappush(ranking,(float(compute_median(short_path[cat])),cat))

100%|██████████| 35/35 [08:18<00:00,  2.10s/it]


In [24]:
n=len(ranking)
block_ranking=[]
for  i in range(n):
    block_ranking.append(heapq.heappop(ranking)[1])

In [25]:
block_ranking.remove('Year_of_birth_unknown')
app=block_ranking
block_ranking=['Year_of_birth_unknown']+ app

Finally this is our $block_{RANKING}$:

In [26]:
block_ranking

['Year_of_birth_unknown',
 'American_film_actors',
 'English-language_films',
 'American_Jews',
 'American_films',
 'American_television_actors',
 'Article_Feedback_Pilot',
 'Black-and-white_films',
 'British_films',
 'English_television_actors',
 'Indian_films',
 'Members_of_the_United_Kingdom_Parliament_for_English_constituencies',
 'English-language_albums',
 'People_from_New_York_City',
 'Rivers_of_Romania',
 'Debut_albums',
 'American_military_personnel_of_World_War_II',
 'Association_football_defenders',
 'Association_football_forwards',
 'Association_football_goalkeepers',
 'Association_football_midfielders',
 'Asteroids_named_for_people',
 'English_cricketers',
 'English_footballers',
 'Fellows_of_the_Royal_Society',
 'Harvard_University_alumni',
 'Living_people',
 'Main_Belt_asteroids',
 'Major_League_Baseball_pitchers',
 'Place_of_birth_missing_(living_people)',
 'The_Football_League_players',
 'Windows_games',
 'Year_of_birth_missing',
 'Year_of_birth_missing_(living_people)

Now since we have the $block_{RANKING}$ vector, we proceed with the sorting of nodes in each category:

### [Step 1]: Compute subgraph induced by $C_0$. For each node compute the sum of the weigths of the in-edges.

$$score_{article_i}=\sum_{i\in in-edges} w_i$$

Using the library *networkx* we compute the full graph, using the dictionary *direct_graph*
after we use the same library for build the subgraph induced by the category $C_0$.

In [27]:
G=nx.DiGraph(direct_graph)

In [28]:
C0=block_ranking[0]
sg=G.subgraph(clean_cat[C0])

In [29]:
scores = defaultdict(int)
for i in list(sg.nodes):
    score = sg.in_degree(i)
    scores[i] = score

In [ ]:
scores

### [Step 2]:  Extend the graph to the nodes that belong to $C_1$. Thus, for each article in $C_1$ compute the score as before. 

In [30]:
C1=block_ranking[1]
sg2=G.subgraph(clean_cat[C0] + clean_cat[C1])

In [31]:
for i in set(sg2.nodes)-set(sg.nodes):
    for j in range(len(list(sg2.in_edges(i)))):
        sv = list(sg2.in_edges(i))[j][0]
        if sv in list(sg.nodes):
            scores[i] += scores[sv]
        else:
            scores[i] += 1

### [Step 3]: Repeat Step2 up to the last category of the ranking. In the last step of the example you clearly see the weight update of the edge coming from node E.

We're interessed to rank the articles in each categories. For sorting the articles using the score, we build the following function, that given a dictionary and a set of keys, it return the list of keys sorted by the value associated to each key. 

In [60]:
def page_ranking(d,s):
    l=[]
    for k in s:
        l.append((k, d[k]))
    sorted_by_second=sorted(l, key=lambda tup: tup[1])
    l_sort=[x[0] for x in sorted_by_second]
    
    return(l_sort)

In [59]:
cat_c = set()
scores_tot = defaultdict(int)
pages=[]
for cat in tqdm(block_ranking):
    set_clean_cat = set(clean_cat[cat])
    considering= set_clean_cat-cat_c
    cat_c = cat_c | set_clean_cat
    sg_tot = G.subgraph(cat_c)
    for i in (considering):
        for j in (list(sg_tot.in_edges(i))):
            sv = j[0]
            if sv in considering:
                scores_tot[i] += 1
            else:
                scores_tot[i] += scores_tot[sv]
    pages+=page_ranking(scores_tot, set_clean_cat)
    

100%|██████████| 35/35 [00:23<00:00,  1.86it/s]


For a nice visualization, we want to print the names of the articles, instead of the number of node. So we build the following dictionary, called *names* that has for keys the ID's of the article and for value the name of the ariticle:

$$\textit{names}=\{\textit{ArticleId}_i:\textit{ArticleName}_i\}$$

In [69]:
names={}
with open('wiki-topcats-page-names.txt') as f: 
    for line in f:
        l=list(line.split())
        names[int(l[0])]=' '.join(l[1::])

We print the first 100 articles:

In [71]:
for i in range(101):
    print(names[pages[i]])

Kisen
Mibu no Tadami
Jean Carol
Conrad (Bishop of Utrecht)
Anna Lehr
Hodo Nivica
Godfrey I, Duke of Lower Lorraine
Hierocles (author of Synecdemus)
Dahteste
Manuel Surez de Begoa
William Rainborowe
Nikos E. Politis
Mary Oxlie
William Fraser (bishop)
Elizabeth Stewart, Countess of Crawford
Walter O'Hara
Thomas Burton (politician)
Archibald Van Horne
Mila Schn
Kzim-i-Samandar
An Collins
Mary Hill, Countess of Hillsborough
Violet Mond, Baroness Melchett
Raphael Neale
Chevalier des Marchais
Heather Lechtman
Pemmasani Ramalinga Nayudu
Varq
Shivappa Nayaka
Nicomachus (scribe)
Alexander Wilson (U.S. Representative)
Johan Danckerts
James Johnson (Virginia congressman)
Cornelio Da Montalcino
William N. Grover
Piper Kerman
Thomas Davenport (congressman)
Caroline Cooke
Martin Parker
Jean-Marie Boisvert
Birgitte Macmillan, Countess of Stockton
Charles Constantine of Vienne
Francis Burleigh
Agrippa the Skeptic
Hilde Holovsky
John Cochran (artist)
Edith Borella
Marcus Manilius
Benjamin Avery
Julius 